In [21]:
import torch
import ray
from ray import tune
from ray.air import session
from sklearn.model_selection import train_test_split
import yaml
import argparse
import shutil
from collections import namedtuple
import os
import datetime


# Custom Libraries
from utils.data_generator import DataGenerator
from utils.agent import TrainModel
from utils.helper import to_python_native, gen_experiment_name, set_seed, save_model_state
from utils.plotter import alignment_progress_over_iterations, plot_initial_final_accuracies
from utils.pennylane.model import Qkernel

In [22]:
# === Backend Configuration ===
if torch.backends.mps.is_available():
    device = torch.device("mps")
elif torch.cuda.is_available():
    device = torch.device("cuda")
elif torch.xpu.is_available():
    device = torch.device("xpu")
else:
    device = torch.device("cpu")

set_seed(42)

In [23]:
data_generator = DataGenerator(
        dataset_name= 'wine',
        file_path=None,
    )

training_data, training_labels, testing_data, testing_labels = data_generator.generate_dataset()
training_data = torch.tensor(training_data.to_numpy(), dtype=torch.float32, requires_grad=True)
testing_data = torch.tensor(testing_data.to_numpy(), dtype=torch.float32, requires_grad=True)
training_labels = torch.tensor(training_labels.to_numpy(), dtype=torch.int)
testing_labels = torch.tensor(testing_labels.to_numpy(), dtype=torch.int)

In [24]:
print(training_data.size())
print(testing_data.size())

torch.Size([142, 13])
torch.Size([36, 13])


In [25]:
kernel = Qkernel(
        device='lightning.qubit',
        n_qubits=4,
        trainable=True,
        input_scaling=True,
        data_reuploading=True,
        ansatz='embedding_paper',
        ansatz_layers=3
    )

In [26]:
agent = TrainModel(
        kernel=kernel,
        training_data=training_data,
        training_labels=training_labels,
        testing_data=testing_data,
        testing_labels=testing_labels,
        optimizer='gd',
        lr= 0.1,
        mclr=0.01,
        cclr=0.01,
        epochs=100,
        train_method='ccka',
        target_accuracy=0.95,
        get_alignment_every=10,
        validate_every_epoch=1,
        base_path='./',
        lambda_kao=0.001,
        lambda_co=0.001,
        clusters=10
    )

Epochs:  10


In [27]:
before_metrics = agent.evaluate(testing_data, testing_labels, 'before')
before_metrics

{'alignment': tensor(2.6561, grad_fn=<SqueezeBackward0>),
 'executions': None,
 'training_accuracy': 0.8943661971830986,
 'testing_accuracy': 0.7777777777777778,
 'f1_score': 0.784795321637427,
 'alignment_arr': [[], []],
 'loss_arr': [],
 'validation_accuracy_arr': []}

In [28]:
agent.fit_multiclass(training_data, training_labels)

Started Training
------------------------------------------------------------------
Epoch: 0 — Alignment per main centroid
Centroid 0 (label=0): Alignment = -0.5721251368522644
Centroid 1 (label=1): Alignment = -0.7101485133171082
Centroid 2 (label=2): Alignment = -0.1201796755194664
------------------------------------------------------------------
------------------------------------------------------------------
Epoch: 1 — Alignment per main centroid
Centroid 0 (label=0): Alignment = -0.5435969233512878
Centroid 1 (label=1): Alignment = -0.6837146878242493
Centroid 2 (label=2): Alignment = -0.052386797964572906
------------------------------------------------------------------
------------------------------------------------------------------
Epoch: 2 — Alignment per main centroid
Centroid 0 (label=0): Alignment = -0.5388357043266296
Centroid 1 (label=1): Alignment = -0.6530607342720032
Centroid 2 (label=2): Alignment = -0.03502922132611275
------------------------------------------

(Qkernel(),
 [Parameter containing:
  tensor([[1.2996, 0.8045, 0.7182, 1.3321],
          [1.2320, 0.9792, 0.8738, 0.9024],
          [1.1460, 2.0529, 0.5664, 1.1596]], requires_grad=True),
  Parameter containing:
  tensor([[ 1.8641,  2.4850, -1.4819,  2.0193],
          [-0.2380,  0.0907, -1.3022,  1.9428],
          [ 2.6082, -2.1705,  2.5716,  0.5537]], requires_grad=True),
  Parameter containing:
  tensor([[ 1.0367,  0.5376,  1.4998,  0.2749],
          [ 2.9003,  0.0324, -1.2780,  0.2352],
          [-1.3631, -0.3470, -1.2017,  1.9627]], requires_grad=True)],
 ParameterList(
     (0): Parameter containing: [torch.float32 of size 13]
     (1): Parameter containing: [torch.float32 of size 13]
     (2): Parameter containing: [torch.float32 of size 13]
 ),
 ParameterList(
     (0): Parameter containing: [torch.float32 of size 13]
     (1): Parameter containing: [torch.float32 of size 13]
     (2): Parameter containing: [torch.float32 of size 13]
     (3): Parameter containing: [torch.

In [30]:
after_metrics = agent.evaluate(testing_data, testing_labels, 'after')
after_metrics

{'alignment': tensor(2.3824, grad_fn=<SqueezeBackward0>),
 'executions': 9000,
 'training_accuracy': 0.7887323943661971,
 'testing_accuracy': 0.6666666666666666,
 'f1_score': 0.674727668845316,
 'alignment_arr': [[], []],
 'loss_arr': [],
 'validation_accuracy_arr': []}